In [2]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, confusion_matrix
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [3]:
# Loading the Excel file
df = pd.read_excel('negative_sentiments.xlsx')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   User           209 non-null    object 
 1   Tweet          209 non-null    object 
 2   Cleaned_Tweet  209 non-null    object 
 3   Negative       209 non-null    float64
 4   Neutral        209 non-null    float64
 5   Positive       209 non-null    float64
 6   Sentiment      209 non-null    object 
 7   Label          151 non-null    float64
dtypes: float64(4), object(4)
memory usage: 13.2+ KB


In [5]:
predict_df = df[df['Label'].isna()]

In [6]:
predict_df

,User,Tweet,Cleaned_Tweet,Negative,Neutral,Positive,Sentiment,Label
151,Bev,@GyAncient @nige_gallop @CCLeeFreeman @Cleeccs...,bagger amazed knew walking football result yes...,0.519025,0.442044,0.038931,negative,NaN
152,Nigel 💙,@GyAncient @BevskiMids @CCLeeFreeman @Cleeccsc...,blimey mick old dog,0.677478,0.285934,0.036588,negative,NaN
153,Nigel 💙,@BevskiMids @CCLeeFreeman @Cleeccsc @Humberbea...,wondered rd right obvs due play rare midweek g...,0.763980,0.222234,0.013785,negative,NaN
154,Iain Joseph Gorry*,@Cleeccsc @JoRobbo68 @Humberbeat @HumbersideFi...,guy tweet date wrong think,0.664363,0.322601,0.013036,negative,NaN
155,North East Lincolnshire Council,What are the biggest crime issues in North Eas...,biggest crime issue north east lincolnshire te...,0.662119,0.320535,0.017346,negative,NaN
156,South Yorkshire Fire,We spent weeks tackling a fire on Hatfield Moo...,spent week tackling fire hatfield moor despera...,0.560314,0.336407,0.103279,negative,NaN
157,Humberside Police - North East Lincolnshire,#Grimsby #Willows Attended an incident tonight...,attended incident tonight binbrook way group y...,0.889538,0.104743,0.005718,negative,NaN
158,Safer Roads Humber,A fire safety message today. With more people ...,fire safety message today people easy overload...,0.514286,0.462393,0.023321,negative,NaN
159,North Lincs Council,"Just because it's warm outside, it doesn't mea...",warm outside doesnt mean warm underwater cold ...,0.698401,0.289254,0.012345,negative,NaN
160,DC_LK1989,Unfortunately I’m going to say no...our street...,unfortunately im going say noour street would ...,0.875694,0.117654,0.006652,negative,NaN


In [7]:
train_df = df.dropna(subset=['Label'])


In [8]:
train_df

,User,Tweet,Cleaned_Tweet,Negative,Neutral,Positive,Sentiment,Label
0,Phillip Marshall,@HumbersideFire No mention of headlights. Disa...,mention headlight disappointing,0.909523,0.084315,0.006161,negative,0.0
1,@KateHull,"Yet another historic, #grade2listed #church up...",yet another historic flame cost u heritage imp...,0.890281,0.105682,0.004037,negative,1.0
2,Abigail Wainwright,@davidthomas4085 @HumbersideFire @East_Riding ...,working hour shame drain cranswick seem poorly...,0.947041,0.049033,0.003926,negative,1.0
3,Phil Shillito,@cllrmikeross Sad news and Mark will be missed...,sad news mark missed many condolence family,0.789475,0.187370,0.023155,negative,0.0
4,Radhotchoco,"@HumbersideFire part of building at Atom bar, ...",part building atom bar princess avenue upstair...,0.730137,0.260935,0.008928,negative,1.0
...,...,...,...,...,...,...,...,...
146,Phillip Norton,The fire at Haltemprice Crematorium is now out...,fire haltemprice crematorium operator cremator...,0.628737,0.359705,0.011559,negative,0.0
147,Theresa Capper,@HumbersideFire Great punishment....NOT!!! 🤬 s...,great punishmentnot shame couldnt turn hose jo...,0.562310,0.319152,0.118538,negative,0.0
148,CDW,@simonpheadley @Stef_Walton @HumbersideFire Th...,fact command unit aerial attendance saying wor...,0.775761,0.213087,0.011153,negative,0.0
149,CDW,@HumbersideFire Guess this has gotten worse fr...,guess gotten worse smoke issuing major fire,0.840872,0.149536,0.009592,negative,1.0


In [9]:
train_df['Label'].value_counts()

Label
0.0    118
1.0     33
Name: count, dtype: int64

# RoBERTa Model

In [10]:
# Tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Tokenizing and preparing DataLoader for training
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])

        label = 0 if self.labels is None else int(self.labels[idx])

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [12]:
# Splitting the training data for RoBERTa
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['Cleaned_Tweet'].values,
    train_df['Label'].values,
    test_size=0.1,
    random_state=42
)

In [13]:
# Creating datasets and DataLoader for training
train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer)

In [14]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [15]:
# Training the RoBERTa 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
epochs = 100

best_val_loss = float('inf')
patience = 7 
counter = 0

for epoch in range(epochs):
    # Training loop
    model.train()
    for batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()

    # Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            loss = torch.nn.functional.cross_entropy(logits, labels)

            val_loss += loss.item()

    # Calculating average validation loss
    avg_val_loss = val_loss / len(val_loader)

    # Checking for early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        counter = 0
    else:
        counter += 1

    if counter >= patience:
        print(f'Early stopping after {epoch + 1} epochs without improvement.')
        break

Validation: 100%|████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  3.93it/s]

Early stopping after 13 epochs without improvement.


In [16]:
# Validation
model.eval()
val_preds = []
val_true = []

with torch.no_grad():
    for batch in tqdm(val_loader, desc="Validation"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        val_preds.extend(preds.cpu().numpy())
        val_true.extend(labels.cpu().numpy())

print(classification_report(val_true, val_preds))

Validation: 100%|████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  7.75it/s]

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        15
           1       0.00      0.00      0.00         1

    accuracy                           0.94        16
   macro avg       0.47      0.50      0.48        16
weighted avg       0.88      0.94      0.91        16




C:\Users\Amro\anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Amro\anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Amro\anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
# Calculating Mean Squared Error (MSE)
mse = mean_squared_error(val_true, val_preds)
print(f"Mean Squared Error (MSE): {mse:.4f}")

# Calculating Mean Absolute Error (MAE)
mae = mean_absolute_error(val_true, val_preds)
print(f"Mean Absolute Error (MAE): {mae:.4f}")

# Calculating R-squared (R2) score
r2 = r2_score(val_true, val_preds)
print(f"R-squared (R2) Score: {r2:.4f}")

Mean Squared Error (MSE): 0.0625
Mean Absolute Error (MAE): 0.0625
R-squared (R2) Score: -0.0667


### Prediction of RoBERTa model on Tweets

In [18]:
# loading file with data for prediction
new_df = pd.read_excel('Labeled_Tweets.xlsx')

In [19]:
# Assuming 'Cleaned_Tweet' is the column containing the text data for prediction
new_texts = new_df['Cleaned_Tweet'].values

# Tokenize and prepare DataLoader for prediction
new_dataset = CustomDataset(new_texts, labels=None, tokenizer=tokenizer, max_len=128)
new_loader = DataLoader(new_dataset, batch_size=8, shuffle=False, collate_fn=lambda x: x)

# Make predictions
model.eval()
predictions = []

with torch.no_grad():
    for batch in tqdm(new_loader, desc="Predicting"):
        input_ids = torch.stack([item['input_ids'] for item in batch]).to(device)
        attention_mask = torch.stack([item['attention_mask'] for item in batch]).to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        predictions.extend(preds.cpu().numpy())

Predicting: 100%|████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00,  8.47it/s]


In [20]:
# Adding predictions to the DataFrame
new_df['RoBERTa_Prediction'] = predictions

In [21]:
new_df

,User,Tweet,Cleaned_Tweet,Negative,Neutral,Positive,Sentiment,Label,RoBERTa_Prediction
0,Bev,@GyAncient @nige_gallop @CCLeeFreeman @Cleeccs...,bagger amazed knew walking football result yes...,0.519025,0.442044,0.038931,negative,0,0
1,Nigel 💙,@GyAncient @BevskiMids @CCLeeFreeman @Cleeccsc...,blimey mick old dog,0.677478,0.285934,0.036588,negative,0,0
2,Nigel 💙,@BevskiMids @CCLeeFreeman @Cleeccsc @Humberbea...,wondered rd right obvs due play rare midweek g...,0.763980,0.222234,0.013785,negative,0,0
3,Iain Joseph Gorry*,@Cleeccsc @JoRobbo68 @Humberbeat @HumbersideFi...,guy tweet date wrong think,0.664363,0.322601,0.013036,negative,0,0
4,North East Lincolnshire Council,What are the biggest crime issues in North Eas...,biggest crime issue north east lincolnshire te...,0.662119,0.320535,0.017346,negative,0,0
5,South Yorkshire Fire,We spent weeks tackling a fire on Hatfield Moo...,spent week tackling fire hatfield moor despera...,0.560314,0.336407,0.103279,negative,1,1
6,Humberside Police - North East Lincolnshire,#Grimsby #Willows Attended an incident tonight...,attended incident tonight binbrook way group y...,0.889538,0.104743,0.005718,negative,1,1
7,Safer Roads Humber,A fire safety message today. With more people ...,fire safety message today people easy overload...,0.514286,0.462393,0.023321,negative,0,0
8,North Lincs Council,"Just because it's warm outside, it doesn't mea...",warm outside doesnt mean warm underwater cold ...,0.698401,0.289254,0.012345,negative,0,0
9,DC_LK1989,Unfortunately I’m going to say no...our street...,unfortunately im going say noour street would ...,0.875694,0.117654,0.006652,negative,0,0


In [22]:
new_df['RoBERTa_Prediction'].value_counts()

RoBERTa_Prediction
0    44
1    14
Name: count, dtype: int64

In [23]:
new_df['Label'].value_counts()

Label
0    46
1    12
Name: count, dtype: int64

In [24]:
# Calculating metrics
accuracy = accuracy_score(new_df['Label'], new_df['RoBERTa_Prediction'])
precision = precision_score(new_df['Label'], new_df['RoBERTa_Prediction'])
recall = recall_score(new_df['Label'], new_df['RoBERTa_Prediction'])
f1 = f1_score(new_df['Label'], new_df['RoBERTa_Prediction'])
roc_auc = roc_auc_score(new_df['Label'], new_df['RoBERTa_Prediction'])

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"AUC-ROC: {roc_auc}")


Accuracy: 0.9310344827586207
Precision: 0.7857142857142857
Recall: 0.9166666666666666
F1 Score: 0.8461538461538461
AUC-ROC: 0.9257246376811593


In [25]:
# Creating a confusion matrix
conf_matrix = confusion_matrix(new_df['Label'], new_df['RoBERTa_Prediction'])

# Extracting TP, TN, FP, FN from the confusion matrix
TP = conf_matrix[1, 1]
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
FN = conf_matrix[1, 0]

# Calculating the number of wrong predictions
wrong_predictions = FP + FN


print(f"Number of wrong predictions: {wrong_predictions}")

Number of wrong predictions: 4


In [26]:
# Sample list of sentences
sentences = ["Had a wonderful time in hull today","there is a fire in the south street we need the your assistance @HumbersideFire", "The kids are lighting fireworks in pearson park it is really dangerous", "I see smoke coming from the paragon station", "Some teenager are jumping of the bridge into the water","There is no incident in the beverly road"]

# Tokenizing the list of sentences
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt').to('cuda')

# Forward pass through the model
outputs = model(**inputs)

# Applying softmax to get predictions
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

# Converting predictions to numpy array
predictions = predictions.cpu().detach().numpy()

# Getting the predicted labels
predicted_labels = [np.argmax(pred) for pred in predictions]

# Getting the corresponding probabilities
probs = [pred[label] for pred, label in zip(predictions, predicted_labels)]


for sentence, label, prob in zip(sentences, predicted_labels, probs):
    print(f"Sentence: {sentence}")
    print(f"Predicted Label: {label}")
    print(f"Probability: {prob:.4f}")
    print()


Sentence: Had a wonderful time in hull today
Predicted Label: 0
Probability: 0.9960

Sentence: there is a fire in the south street we need the your assistance @HumbersideFire
Predicted Label: 1
Probability: 0.9957

Sentence: The kids are lighting fireworks in pearson park it is really dangerous
Predicted Label: 1
Probability: 0.9976

Sentence: I see smoke coming from the paragon station
Predicted Label: 1
Probability: 0.9989

Sentence: Some teenager are jumping of the bridge into the water
Predicted Label: 1
Probability: 0.9652

Sentence: There is no incident in the beverly road
Predicted Label: 1
Probability: 0.9873

